## 生长培养基

营养物质的可用性对代谢通量有重大影响，`cobrapy`为管理外部环境与代谢模型之间的交换提供了一些帮助。在实验环境中，“环境”通常是由生长介质构成的，因此，所有代谢物和辅助因子的浓度都是模型生物可用的。然而，基于约束的代谢模型只考虑通量。因此，不能简单地使用浓度，因为通量的单位是`mmol / [gDW h]` 每克细胞干重和每小时的浓度)。
此外，您正在为特定导入通量设置上限，而不是为通量本身设置上限。这里有一些粗略的近似。例如，如果你每24小时向1克细菌提供1摩尔葡萄糖，你可以将葡萄糖的最高交换通量设置为1摩尔/`1 mol / [1 gDW * 24 h]` ，因为这是可以进口的名义最大值。但不能保证葡萄糖会随流量被消耗。因此，交换通量的优选数据是直接通量测量，例如从时间过程外代谢组测量中获得的数据。
那么在 COBRApy 中是什么样子的呢？模型的当前增长介质由“medium”属性管理。

In [1]:
from cobra.io import load_model

model = load_model("textbook")
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

这将返回一个包含所有活动交换通量(具有非零通量边界的交换通量)的通量上限字典。现在我们看到我们已经实现了有氧生长。您可以通过将字典分配给模型来修改模型的生长介质。`model.medium` ，将交换反应映射到各自的进口上限。现在让我们通过关闭氧气输入来强制厌氧生长。

In [2]:
medium = model.medium
medium["EX_o2_e"] = 0.0
model.medium = medium

model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

正如我们所看到的，氧气进口现在从活跃的交换列表中删除，我们可以验证这也导致了较低的增长率。

In [3]:
model.slim_optimize()

0.21166294973531258

这里有个小陷阱。 因为`model.medium`不能直接分配，所以以下操作将不起作用:

In [4]:
model.medium["EX_co2_e"] = 0.0
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

正如你所看到的，`EX_co2_e`没有设置为零。这是因为model.medium只是当前交换通量的副本。直接用`model.medium[...] = ...` 不会改变模式。你必须用改变后的导入通量上界来分配整个字典:

In [5]:
medium = model.medium
medium["EX_co2_e"] = 0.0
model.medium = medium

model.medium  # now it worked

{'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

设置生长介质还连接到上下文管理器，因此您可以以可逆的方式设置特定的生长介质。

In [6]:
model = load_model("textbook")

with model:
    medium = model.medium
    medium["EX_o2_e"] = 0.0
    model.medium = medium
    print(model.slim_optimize())
print(model.slim_optimize())
model.medium

0.21166294973531258
0.8739215069684315


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

因此，介质变化仅在`with`块内应用并自动恢复。

## 最小培养基

在某些情况下，您可能对能够保持特定生长速率的最小生长介质感兴趣，即所谓的最小介质。为此，我们提供了minimal_medium`函数，默认情况下获得具有最低总进口通量的介质。这个函数需要两个参数:模型和其必须达到的最小增长率(或其他目标)。

In [15]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth)

d:\ProgramData\anaconda3\Lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


EX_glc__D_e    10.000000
EX_nh4_e        4.765319
EX_o2_e        21.799493
EX_pi_e         3.214895
dtype: float64

所以我们看到生长实际上受到葡萄糖输入的限制。
或者，您可能对具有最少数量的活动导入的最小介质感兴趣。这可以通过使用`minimize_components`参数来实现(注意，这使用了MIP公式，因此会慢得多)。

In [8]:
minimal_medium(model, 0.1, minimize_components=True)

d:\ProgramData\anaconda3\Lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


EX_glc__D_e    10.00000
EX_nh4_e        0.54528
EX_pi_e         0.36787
dtype: float64

在尽量减少进口通量的数量时，可能有许多替代解决方案。为了得到其中的几个，你也可以传递一个正整数来`minimize_components`，这将给你最多50个备选解。让我们在我们的模型中尝试一下，并使用 `open_exchanges` 参数，该参数将为模型中的所有进口反应分配一个大的上界。返回类型将是`pandas.DataFrame`。

In [9]:
minimal_medium(model, 0.8, minimize_components=8, open_exchanges=True)

d:\ProgramData\anaconda3\Lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()
d:\ProgramData\anaconda3\Lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()
d:\ProgramData\anaconda3\Lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()
d:\ProgramData\anaconda3\Lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explici

,0,1,2,3,4,5
EX_fru_e,0.000000,308.82944,0.00000,0.000000,0.000000,0.000000
EX_glc__D_e,0.000000,0.00000,308.82944,31.084608,502.051914,0.000000
EX_gln__L_e,0.000000,0.00000,0.00000,2.181120,0.000000,0.000000
EX_glu__L_e,346.141716,4.36224,4.36224,0.000000,0.000000,24.798770
EX_mal__L_e,0.000000,0.00000,0.00000,0.000000,0.000000,1000.000000
EX_nh4_e,0.000000,0.00000,0.00000,0.000000,8.417918,0.000000
EX_o2_e,500.000000,0.00000,0.00000,0.000000,0.000000,0.000000
EX_pi_e,65.104125,2.94296,2.94296,2.942960,5.679100,15.546299


所以总共有4个解。一个好氧和三个厌氧使用不同的碳源。

## 边界反应
除交换反应外，还有其他类型的边界反应，如需求反应或吸收反应。`cobrapy` 使用各种启发式方法来识别它们，并且可以通过使用适当的属性来访问它们。对于交换反应:

In [10]:
ecoli = load_model("iJO1366")
ecoli.exchanges[0:5]

[<Reaction EX_12ppd__R_e at 0x21342599f10>,
 <Reaction EX_12ppd__S_e at 0x21342576390>,
 <Reaction EX_14glucan_e at 0x21342576350>,
 <Reaction EX_15dap_e at 0x21342576750>,
 <Reaction EX_23camp_e at 0x21342576b10>]

对于需求反应:

In [11]:
ecoli.demands

[<Reaction DM_4crsol_c at 0x21342573a50>,
 <Reaction DM_5drib_c at 0x21342573bd0>,
 <Reaction DM_aacald_c at 0x21342574450>,
 <Reaction DM_amob_c at 0x21342574810>,
 <Reaction DM_mththf_c at 0x21342574c10>,
 <Reaction DM_oxam_c at 0x21342574fd0>]

对于汇反应:

In [16]:
ecoli.sinks

[]

所有边界反应(任何消耗或引入质量到系统中的反应)都可以用`boundary`属性获得:

In [13]:
ecoli.boundary[0:10]

[<Reaction DM_4crsol_c at 0x21342573a50>,
 <Reaction DM_5drib_c at 0x21342573bd0>,
 <Reaction DM_aacald_c at 0x21342574450>,
 <Reaction DM_amob_c at 0x21342574810>,
 <Reaction DM_mththf_c at 0x21342574c10>,
 <Reaction DM_oxam_c at 0x21342574fd0>,
 <Reaction EX_12ppd__R_e at 0x21342599f10>,
 <Reaction EX_12ppd__S_e at 0x21342576390>,
 <Reaction EX_14glucan_e at 0x21342576350>,
 <Reaction EX_15dap_e at 0x21342576750>]